In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# 한글 font 설정
import matplotlib.font_manager as fm

font_name = fm.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
plt.rc('font',family = font_name)
mpl.rcParams['axes.unicode_minus'] = False

# 컬럼을 다 보여주는 함수
from IPython.display import display
pd.options.display.max_columns = None

import re
from sklearn.model_selection import train_test_split

## 설문 응답 결과 csv 파일로 만들어 파이썬에 로드

1. 응답 결과 106개 데이터: 데이터셋 수 너무 적어 머신러닝 제대로 구현 안될 수 있다

In [2]:
cycle = pd.read_csv("자전거 구입 선호 조사(응답).csv")
cycle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 11 columns):
Unnamed: 0                          0 non-null float64
당신의 성별은 무엇입니까?                      106 non-null object
당신의 연령대는 어떻게 되십니까?                  105 non-null object
자전거를 얼마나 보유하고 계십니까?                 106 non-null object
어떤 자전거를 보유하고 계십니까?                  98 non-null object
주로 타는 자전거는 어떤 종류입니까?                98 non-null object
주로 어떤 목적으로 자전거를 이용하십니까?             97 non-null object
일주일에 얼마나 라이딩을 하십니까?                 98 non-null object
한 번에 얼마나 라이딩을 하십니까?                 98 non-null object
자전거를 구입하신다면, 어떤 자전거를 구입할 계획이십니까?    101 non-null object
어떤 목적으로 자전거를 구입할 계획이십니까?            101 non-null object
dtypes: float64(1), object(10)
memory usage: 9.2+ KB


In [3]:
cycle.head()

,Unnamed: 0,당신의 성별은 무엇입니까?,당신의 연령대는 어떻게 되십니까?,자전거를 얼마나 보유하고 계십니까?,어떤 자전거를 보유하고 계십니까?,주로 타는 자전거는 어떤 종류입니까?,주로 어떤 목적으로 자전거를 이용하십니까?,일주일에 얼마나 라이딩을 하십니까?,한 번에 얼마나 라이딩을 하십니까?,"자전거를 구입하신다면, 어떤 자전거를 구입할 계획이십니까?",어떤 목적으로 자전거를 구입할 계획이십니까?
0,NaN,남성,30대,1대,로드,로드,레저,주 1~2회,1~2시간,그래블,레저
1,NaN,남성,30대,2대,"로드, 미니벨로",로드,레저,주 1~2회,1~2시간,구입x,구입x
2,NaN,남성,30대,보유하고 있지 않다,NaN,NaN,NaN,NaN,NaN,로드,레저
3,NaN,남성,30대,2대,"생활 자전거, MTB","생활 자전거, MTB",출퇴근(이동 수단),주 3~4회,3~4시간,생활 자전거,로드 멘붕
4,NaN,남성,40대,보유하고 있지 않다,NaN,NaN,NaN,NaN,NaN,MTB,레저


## 컬럼명 알기 쉽도록 명사화

In [4]:
cycle.rename(columns={"당신의 성별은 무엇입니까?":"성별", 
                      "당신의 연령대는 어떻게 되십니까?":"연령", 
                      "자전거를 얼마나 보유하고 계십니까?":"보유 수",
                      "주로 타는 자전거는 어떤 종류입니까?":"선호 자전거",
                      "주로 어떤 목적으로 자전거를 이용하십니까?":"목적",
                      "일주일에 얼마나 라이딩을 하십니까?":"라이딩 빈도(일주)",
                      "한 번에 얼마나 라이딩을 하십니까?":"라이딩 시간",
                      "자전거를 구입하신다면, 어떤 자전거를 구입할 계획이십니까?":"구입할 자전거",
                      "어떤 목적으로 자전거를 구입할 계획이십니까?":"구입 목적"
                     }, inplace=True)

In [5]:
cycle.rename(columns={"어떤 자전거를 보유하고 계십니까?":"보유 자전거"}, inplace=True)

In [6]:
cycle

,Unnamed: 0,성별,연령,보유 수,보유 자전거,선호 자전거,목적,라이딩 빈도(일주),라이딩 시간,구입할 자전거,구입 목적
0,NaN,남성,30대,1대,로드,로드,레저,주 1~2회,1~2시간,그래블,레저
1,NaN,남성,30대,2대,"로드, 미니벨로",로드,레저,주 1~2회,1~2시간,구입x,구입x
2,NaN,남성,30대,보유하고 있지 않다,NaN,NaN,NaN,NaN,NaN,로드,레저
3,NaN,남성,30대,2대,"생활 자전거, MTB","생활 자전거, MTB",출퇴근(이동 수단),주 3~4회,3~4시간,생활 자전거,로드 멘붕
4,NaN,남성,40대,보유하고 있지 않다,NaN,NaN,NaN,NaN,NaN,MTB,레저
5,NaN,남성,50대,1대,MTB,MTB,레저,주 1~2회,3~4시간,MTB,레저
6,NaN,남성,40대,2대,"로드, MTB",로드,레저,주 3~4회,3~4시간,로드,레저
7,NaN,남성,30대,2대,로드,로드,출퇴근(이동 수단),주 3~4회,1시간 미만,로드,레저
8,NaN,남성,20대,2대,"MTB, 하이브리드",MTB,출퇴근(이동 수단),주 1~2회,1~2시간,MTB,여행
9,NaN,남성,40대,1대,하이브리드,하이브리드,출퇴근(이동 수단),주 3~4회,1시간 미만,MTB,레저


## 결측치 확인

In [7]:
cycle["성별"].isnull().mean()

0.0

In [8]:
cycle["연령"].isnull().mean()

0.009433962264150943

In [9]:
cycle["보유 수"].isnull().mean()

0.0

In [10]:
cycle["목적"].isnull().mean()

0.08490566037735849

In [11]:
cycle["구입할 자전거"].isnull().mean()

0.04716981132075472

## 데이터값 변환

1. 머신러닝 구현 위해 데이터값 숫자형으로 변환 

2. 결측치 -1로 대체

In [12]:
cycle["성별"] = cycle["성별"].map({"남성":1, "여성":2})
cycle["연령"] = cycle["연령"].map({"10대":10, "20대":20, "30대":30, "40대":40, "50대":50, "60대 이상":60})
cycle["보유 수"] = cycle["보유 수"].map({"보유하고 있지 않다":0, "1대":1, "2대":2, "3대 이상":3})
cycle["목적"] = cycle["목적"].map({"출퇴근(이동수단)":1, "레저":2, "여행":3, "기타":4})
cycle["구입할 자전거"]  = cycle["구입할 자전거"].map({"생활 자전거":1, "로드":2, "MTB":3, "하이브리드":4, "픽시":5, "미니벨로":6, "기타":7})

cycle["연령"] = cycle["연령"].fillna(-1)
cycle["목적"] = cycle["목적"].fillna(-1)
cycle["구입할 자전거"] = cycle["구입할 자전거"].fillna(-1)

In [13]:
cycle

,Unnamed: 0,성별,연령,보유 수,보유 자전거,선호 자전거,목적,라이딩 빈도(일주),라이딩 시간,구입할 자전거,구입 목적
0,NaN,1,30.0,1,로드,로드,2.0,주 1~2회,1~2시간,-1.0,레저
1,NaN,1,30.0,2,"로드, 미니벨로",로드,2.0,주 1~2회,1~2시간,-1.0,구입x
2,NaN,1,30.0,0,NaN,NaN,-1.0,NaN,NaN,2.0,레저
3,NaN,1,30.0,2,"생활 자전거, MTB","생활 자전거, MTB",-1.0,주 3~4회,3~4시간,1.0,로드 멘붕
4,NaN,1,40.0,0,NaN,NaN,-1.0,NaN,NaN,3.0,레저
5,NaN,1,50.0,1,MTB,MTB,2.0,주 1~2회,3~4시간,3.0,레저
6,NaN,1,40.0,2,"로드, MTB",로드,2.0,주 3~4회,3~4시간,2.0,레저
7,NaN,1,30.0,2,로드,로드,-1.0,주 3~4회,1시간 미만,2.0,레저
8,NaN,1,20.0,2,"MTB, 하이브리드",MTB,-1.0,주 1~2회,1~2시간,3.0,여행
9,NaN,1,40.0,1,하이브리드,하이브리드,-1.0,주 3~4회,1시간 미만,3.0,레저


## 데이터 훈련

In [14]:
X = cycle[["성별", "연령", "보유 수", "목적"]]
y = cycle["구입할 자전거"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [15]:
### KNN

from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier()
KNN.fit(X_train, y_train)

pred=KNN.predict(X_test)

print("훈련 정확도: ", KNN.score(X_train, y_train))
print("예측 정확도: ", KNN.score(X_test, y_test))

훈련 정확도:  0.5443037974683544
예측 정확도:  0.3333333333333333


In [16]:
### 선형회귀

from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

print("훈련 정확도: ", lr.score(X_train, y_train))
print("예측 정확도: ", lr.score(X_test, y_test))

훈련 정확도:  0.07405669958675487
예측 정확도:  0.2869189939897637


In [17]:
### Ridge

from sklearn.linear_model import Ridge

ridge01 = Ridge(alpha=0.1)
ridge01.fit(X_train, y_train)

ridge = Ridge()
ridge.fit(X_train, y_train)

ridge10 = Ridge(alpha=10)
ridge10.fit(X_train, y_train)


print("훈련 정확도: ", ridge01.score(X_train, y_train))
print("예측 정확도: ", ridge01.score(X_test, y_test))

print("훈련 정확도: ", ridge.score(X_train, y_train))
print("예측 정확도: ", ridge.score(X_test, y_test))

print("훈련 정확도: ", ridge10.score(X_train, y_train))
print("예측 정확도: ", ridge10.score(X_test, y_test))

훈련 정확도:  0.0740527172901353
예측 정확도:  0.2851279162979472
훈련 정확도:  0.07372036745740573
예측 정확도:  0.2695306907321311
훈련 정확도:  0.06365789868486826
예측 정확도:  0.164208798736999


In [18]:
### 의사결정나무

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_train, y_train)

print("훈련 정확도: ", tree.score(X_train, y_train))
print("테스트 정확도: ", tree.score(X_test, y_test))

훈련 정확도:  0.5569620253164557
테스트 정확도:  0.5185185185185185


In [22]:
print(tree.feature_importances_)

[0.2654178  0.23338544 0.50119676 0.        ]


In [19]:
### 랜덤포레스트

from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=3, random_state=1)
forest.fit(X_train, y_train)

print("훈련 정확도: ", forest.score(X_train, y_train))
print("테스트 정확도: ", forest.score(X_test, y_test))

훈련 정확도:  0.6582278481012658
테스트 정확도:  0.4074074074074074


# 반성

1. 데이터 수가 너무 적어 제대로 측정 및 예측되지 않음

2. 머신러닝 구현 위한 데이터 정제에 대해 좀더 공부 필요
    1) 이런 식으로 정제가 이루어져도 되는지에 대한 확신 없음
    
    2) 자전거 종류 값을 숫자형으로 변환할 때 의미 없는 숫자형으로 변환 필요
    
    3) 다른 컬럼들도 머신러닝 구현 특성에 포함
    
3. SVM 알고리즘(데이터 수 적어도 머신러닝 구현 가능한 알고리즘) 공부 후 다시 실행 필요

4. 목적에 맞게 설문 문항 세분화 및 더 많은 설문 응답 확보 필요